In [1]:
#---- Tensorflow

import tensorflow as tf

a = tf.placeholder("float")
b = tf.placeholder("float")

y = tf.multiply(a,b)

sess = tf.Session()

print(sess.run(y, feed_dict = {a:3, b:3}))


9.0


In [2]:
#---- mnist_cnn

'''Train a simple convnet on the MNIST dataset.
Run on GPU: THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python mnist_cnn.py
'''

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import json

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

batch_size = 128
nb_classes = 10
nb_epoch = 20

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 3
# convolution kernel size
nb_conv = 5
# size of pooling area for max pooling
nb_pool = 2

# the data, shuffled and split between tran and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()


Using TensorFlow backend.


In [3]:

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:

model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])


C:\Users\Kimsarah\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (5, 5), input_shape=(28, 28, 1..., padding="valid")`
C:\Users\Kimsarah\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (5, 5))`


In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 3)         78        
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 3)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 3)         228       
_________________________________________________________________
activation_2 (Activation)    (None, 20, 20, 3)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 3)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 3)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 300)               0         
__________

In [7]:
hist = model.fit(X_train, Y_train, batch_size=batch_size, 
                 nb_epoch=nb_epoch, verbose=1, validation_split=0.2)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


C:\Users\Kimsarah\Anaconda3\lib\site-packages\keras\models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 32s - loss: 0.3080 - acc: 0.9056 - val_loss: 0.1425 - val_acc: 0.9578
Epoch 2/20
48000/48000 [==============================] - 31s - loss: 0.2688 - acc: 0.9189 - val_loss: 0.1277 - val_acc: 0.9628
Epoch 3/20
48000/48000 [==============================] - 30s - loss: 0.2460 - acc: 0.9272 - val_loss: 0.1158 - val_acc: 0.9660
Epoch 4/20
48000/48000 [==============================] - 30s - loss: 0.2231 - acc: 0.9341 - val_loss: 0.1073 - val_acc: 0.9692
Epoch 5/20
48000/48000 [==============================] - 30s - loss: 0.2080 - acc: 0.9371 - val_loss: 0.1008 - val_acc: 0.9707
Epoch 6/20
48000/48000 [==============================] - 31s - loss: 0.1968 - acc: 0.9399 - val_loss: 0.0965 - val_acc: 0.9712
Epoch 7/20
48000/48000 [==============================] - 30s - loss: 0.1889 - acc: 0.9426 - val_loss: 0.0914 - val_acc: 0.9738
Epoch 8/20
48000/48000 [==============================

In [8]:
import os
os.chdir('C:/Users/Kimsarah/Desktop/SK_ML/code')
# Save the model and weights
json_string = model.to_json()
open('mnist_model_architecture.json','w').write(json_string)
model.save_weights('mnist_model_weights.h5')

# Save History
with open('mnist_model_history.json','w') as fp:
    json.dump(hist.history, fp)
